In [1]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem

import trimesh
import numpy as np
import matplotlib.pyplot as plt

In [2]:
iMols = []
for i in range(1000):
    iMol = Chem.MolFromSmiles('Cc1c(Cl)cccc1Nc1ccccc1C(=O)O')
    iMol = Chem.AddHs(iMol)
    ps = AllChem.ETKDGv3()
    ps.randomSeed = -1
    ps.maxAttempts = 1
    ps.numThreads = 0
    ps.useRandomCoords = True
    re = AllChem.EmbedMultipleConfs(iMol, numConfs = 1, params = ps)
    re = AllChem.MMFFOptimizeMoleculeConfs(iMol,  numThreads = 0)
    iMols.append(iMol)
#Draw.DrawingOptions.bondLineWidth = 100
#draw = Draw.MolToFile(iMol, 'mol.png', size=(600, 600),)

In [3]:
C = []
for i in np.linspace(0,1,len(iMols)):
    C.append(list(plt.cm.rainbow(i))[:3])

In [4]:
axes = trimesh.creation.axis(axis_length=14)
box = trimesh.primitives.Box(extents=(6, 6, 6)).as_outline()
box.colors = np.array([[0, 0, 0]] * len(box.entities))
objects = [axes, box]

In [5]:
c = 0
for iMol in iMols:
    This_mol_Coordinate = []
    for atom in iMol.GetAtoms():
        Coord = list(iMol.GetConformer().GetAtomPosition(atom.GetIdx()))
        This_mol_Coordinate.append(Coord)

    segs = []
    for bond in iMol.GetBonds():
        A = bond.GetBeginAtomIdx()
        B = bond.GetEndAtomIdx()
        segs.append([This_mol_Coordinate[A],This_mol_Coordinate[B]])
    
    segs = trimesh.load_path(segs)
    segs.colors = np.array([C[c]] * len(segs.entities))
    objects.append(segs)
    c += 1

In [6]:
trimesh.Scene(objects).show()

In [7]:
scene = trimesh.scene.Scene(objects)

# a 45 degree homogeneous rotation matrix around
# the Y axis at the scene centroid
rotate = trimesh.transformations.rotation_matrix(
angle=np.radians(10.0),
direction=[0, 1, 0],
point=scene.centroid)

for i in range(1):
    #trimesh.constants.log.info('Saving image %d', i)

    # rotate the camera view transform
    camera_old, _geometry = scene.graph[scene.camera.name]
    camera_new = np.dot(rotate, camera_old)

    # apply the new transform
    scene.graph[scene.camera.name] = camera_new

    # saving an image requires an opengl context, so if -nw
    # is passed don't save the image
    try:
        # increment the file name
        file_name = 'CID610479_number1000.png'
        # save a render of the object as a png
        #7680×4320
        png = scene.save_image(resolution=[11520, 6480], visible=True)
        with open(file_name, 'wb') as f:
            f.write(png)
            f.close()

    except BaseException as E:
        print("unable to save image", str(E))

C:\Users\yxliao\anaconda3\envs\sig\lib\site-packages\pyglet\image\codecs\wic.py:406: UserWarning: [WinError -2147417850] 无法在设置线程模式后对其加以更改。
  warnings.warn(str(err))
